In [2]:
import pennylane as qml
import numpy as np
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

# 1. 定义量子设备（单量子比特，仿真环境）
dev = qml.device("default.qubit", wires=1)

# 2. 定义量子态制备函数（输入参数θ，输出1-保真度（损失函数））
@qml.qnode(dev)
def quantum_state_preparation(theta):
    # 量子门操作：RY门旋转θ角度，从|0⟩制备量子态
    qml.RY(theta, wires=0)
    # 返回制备态的密度矩阵（用于计算保真度）
    return qml.density_matrix(wires=0)

# 3. 定义损失函数（1 - 保真度，目标：最小化该值）
# 修正点1：去掉 theta[0]，直接使用 theta（因为传入的是单个float值）
def loss_function(theta):
    # 目标量子态：|1⟩态的密度矩阵
    target_state = np.array([[0, 0], [0, 1]])
    # 实际制备态的密度矩阵（直接传入 theta，无需下标）
    prepared_state = quantum_state_preparation(theta)
    # 计算保真度（量子态相似度）
    fidelity = qml.math.fidelity(prepared_state, target_state)
    # 返回损失值（1 - 保真度）
    return 1 - fidelity

# 4. 定义量子参数空间（θ∈[0, π]，RY门旋转π角度可从|0⟩到|1⟩）
parameter_space = [Real(low=0, high=np.pi, name="theta")]

# 5. 贝叶斯优化（高斯过程代理模型，期望改进采集函数）
# 修正点2：装饰器处理后，theta 是单个float，直接传入 loss_function 即可
@use_named_args(parameter_space)
def objective(theta):
    return loss_function(theta)

# 运行贝叶斯优化，迭代20次（量子场景足够收敛）
result = gp_minimize(
    objective,
    parameter_space,
    n_calls=20,
    random_state=42,
    acq_func="EI"  # 期望改进，适配量子态制备优化
)

# 6. 输出结果
print("最优旋转角度θ：", result.x[0])
print("最小损失（1-保真度）：", result.fun)
print("对应保真度：", 1 - result.fun)

# 7. 验证最优参数（重新运行量子仿真）
optimal_theta = result.x[0]
final_state = quantum_state_preparation(optimal_theta)
final_fidelity = qml.math.fidelity(final_state, np.array([[0, 0], [0, 1]]))
print("验证：最优参数对应的保真度：", final_fidelity)

最优旋转角度θ： 3.141592653589793
最小损失（1-保真度）： 0.0
对应保真度： 1.0
验证：最优参数对应的保真度： 1.0


In [22]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

# ========== 1. 读取 Excel ==========
df = pd.read_excel("生活检测数据.xlsx")

In [23]:
# 特征（输入）
X = df[[
    "昨晚睡了多久",
    "今天刷手机时长",
    "今天学习时长",
    "今天是否喝咖啡（0/1）",  # 和表格列名完全一致
    "今天压力等级（1–5）"     # 和表格列名完全一致
]]

# 标签（输出）
y = df["第二天状态（0差/1好）"]  # 这里也需要注意，原代码里的"第二天状态"也少了括号部分

print(X)
print(y)


   昨晚睡了多久  今天刷手机时长  今天学习时长  今天是否喝咖啡（0/1）  今天压力等级（1–5）
0       7        2       7             0            1
1       6        3       8             0            5
2       6        2       8             0            4
0    1.0
1    0.0
2    NaN
Name: 第二天状态（0差/1好）, dtype: float64


In [24]:


# ========== 2. 分离特征和标签 ==========
features = [
    "昨晚睡了多久",
    "今天刷手机时长",
    "今天学习时长",
    "今天是否喝咖啡（0/1）", 
    "今天压力等级（1–5）"     
]

# 只用「有第二天状态的数据」来训练
train_df = df.dropna(subset=["第二天状态（0差/1好）"])

X = train_df[features]
y = train_df["第二天状态（0差/1好）"]

# ========== 3. 训练模型 ==========
model = LogisticRegression()
model.fit(X, y)

# ========== 4. 取“今天”的数据 ==========
today = df.iloc[-1]  # 最后一行 = 今天

X_today = today[features].values.reshape(1, -1)

# ========== 5. 预测 ==========
pred = model.predict(X_today)[0]
prob = model.predict_proba(X_today)[0][1]

# ========== 6. 给建议 ==========
print("📅 日期：", today["日期"])
print(f"📊 明天状态好的概率：{prob:.2f}")

if prob >= 0.7:
    advice = "状态很稳，正常作息即可 😄"
elif prob >= 0.4:
    advice = "状态一般，建议今晚早点休息 🙂"
else:
    advice = "状态偏差，今晚强烈建议早点睡 😴"

print("🧠 建议：", advice)


📅 日期： 2026-01-29 00:00:00
📊 明天状态好的概率：0.32
🧠 建议： 状态偏差，今晚强烈建议早点睡 😴


C:\Users\qjx\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\qjx\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
